# Lab 4 - Multiword expressions
Jakub Darul 

In [13]:
from functional import seq, pseq
from fn import _
from typing import List, Set, Tuple, Dict
import pickle
from collections import Counter
import os
import nltk
from nltk.collocations import BigramCollocationFinder
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jakubdarul/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

I downloaded https://raw.githubusercontent.com/bieli/stopwords/master/polish.stopwords.txt. That will prove to be useful later.

In [1]:
sw_file = open("polish.stopwords.txt")
stopwords = set(word[:-1] for word in sw_file.readlines())
sw_file.close()
list(stopwords)[:10]

FileNotFoundError: [Errno 2] No such file or directory: 'lab8/polish.stopwords.txt'

Punctuation was removed and all documents were lowercased before using program written in Rust.

In [15]:
lower_dir = "../lower_ustawy"

In [16]:
def filtering_fun(bigram:Tuple[Tuple[str,str],int])-> bool:
    #At first I did not remove stopwords.
     return bigram[0][0].isalpha() \
        and bigram[0][1].isalpha()\
        and bigram[0][0] not in stopwords\
        and bigram[0][1] not in stopwords

def digrams_per_file(filename:str): #-> Sequence[Tuple[Tuple[str,str],int]]:
    f = open(lower_dir + "/" + filename)
    content = f.read()
    f.close()
    tokens = seq(nltk.word_tokenize(content))
    finder = BigramCollocationFinder.from_words(tokens)
    finder.apply_freq_filter(2) # remove bigrams ocurring only once to save performance
    return seq(finder.ngram_fd.items()).filter(filtering_fun)

In [17]:
def counts(fn_seq):
    counter = Counter()
    def add_digram(bc):
        counter[bc[0]] += bc[1]
    fn_seq.map(digrams_per_file)\
    .for_each(lambda digrams: digrams\
              .for_each(add_digram)
             )
    return counter

In [20]:
%%time
#Here I grouped files by years to parallelize things
bigram_counter = pseq(os.listdir(lower_dir))\
.group_by(lambda fn: fn.split("_")[0])\
.map(lambda x: counts(seq(x[1])))\
.reduce(lambda c1,c2: c1+c2)

bigram_counter

CPU times: user 591 ms, sys: 94 ms, total: 685 ms
Wall time: 14.4 s


In [22]:
%%time
bigram_counter = counts(seq(os.listdir(lower_dir)))

bigram_counter

CPU times: user 27.6 s, sys: 337 ms, total: 27.9 s
Wall time: 28.7 s


We can see 2fold speedup using parallelisation

In [24]:
bigram_counter

Counter({('r', 'nr'): 17734,
         ('państwowej', 'straży'): 491,
         ('straży', 'pożarnej'): 464,
         ('straży', 'granicznej'): 886,
         ('otrzymuje', 'brzmienie'): 9454,
         ('straż', 'graniczną'): 47,
         ('regulacji', 'rynku'): 23,
         ('rynku', 'mleka'): 37,
         ('przetworów', 'mlecznych'): 115,
         ('dostawca', 'bezpośredni'): 13,
         ('jednostkę', 'organizacyjną'): 93,
         ('produkcję', 'mleka'): 14,
         ('gospodarstwie', 'rolnym'): 50,
         ('obrotu', 'mleko'): 5,
         ('przetwory', 'mleczne'): 18,
         ('mleczne', 'przeznaczone'): 3,
         ('bezpośredniego', 'spożycia'): 20,
         ('indywidualna', 'reprezentatywna'): 5,
         ('reprezentatywna', 'zawartość'): 5,
         ('zawartość', 'tłuszczu'): 17,
         ('tłuszczu', 'średnią'): 2,
         ('średnią', 'ważoną'): 3,
         ('ważoną', 'zawartość'): 2,
         ('dostawcę', 'hurtowego'): 21,
         ('dodatków', 'smakowych'): 6,
         ('ko

In [25]:
#Saving bigram counts to use in rust ;)
f = open("bigrams_dict.out","wb")
pickle.dump(dict(bigram_counter),f)
f.close()

# 2-3 Pointwise Mutual Information

Without removing stopwords:

```
PmiEntry(w1: "auschwitz",	w2: "birkenau",	pmi: 19.8375
PmiEntry(w1: "biegłym",	w2: "rewidentom",	pmi: 19.8375
PmiEntry(w1: "bilet",	w2: "pasażerski",	pmi: 19.8375
PmiEntry(w1: "brzeczki",	w2: "miodowej",	pmi: 19.8375
PmiEntry(w1: "budże",	w2: "towym",	pmi: 19.8375
PmiEntry(w1: "butelka",	w2: "miarowa",	pmi: 19.8375
PmiEntry(w1: "będę",	w2: "strzec",	pmi: 19.8375
PmiEntry(w1: "chemioterapii",	w2: "niestandardowej",	pmi: 19.8375
PmiEntry(w1: "cukrem",	w2: "buraczanym",	pmi: 19.8375
PmiEntry(w1: "cyjanoguan",	w2: "cyjanoguanidynę",	pmi: 19.8375
PmiEntry(w1: "damskich",	w2: "męskich",	pmi: 19.8375
PmiEntry(w1: "destylatu",	w2: "winnego",	pmi: 19.8375
PmiEntry(w1: "dokumentacje",	w2: "hydrologiczne",	pmi: 19.8375
PmiEntry(w1: "dotacją",	w2: "podmiotową",	pmi: 19.8375
PmiEntry(w1: "dur",	w2: "brzuszny",	pmi: 19.8375
PmiEntry(w1: "ekrany",	w2: "kinowe",	pmi: 19.8375
PmiEntry(w1: "elektrycznego",	w2: "profilowania",	pmi: 19.8375
PmiEntry(w1: "endo",	w2: "etenooripawina",	pmi: 19.8375
PmiEntry(w1: "ewidencjami",	w2: "paszportowymi",	pmi: 19.8375
PmiEntry(w1: "foul",	w2: "brood",	pmi: 19.8375
PmiEntry(w1: "fundacjom",	w2: "stowarzyszeniom",	pmi: 19.8375
PmiEntry(w1: "fuzji",	w2: "protoplastów",	pmi: 19.8375
PmiEntry(w1: "haemorrhagic",	w2: "septicaemia",	pmi: 19.8375
PmiEntry(w1: "igrzyska",	w2: "olimpijskie",	pmi: 19.8375
PmiEntry(w1: "infrastruktura",	w2: "badawcza",	pmi: 19.8375
PmiEntry(w1: "instrukcyjno",	w2: "metodycznej",	pmi: 19.8375
PmiEntry(w1: "jaja",	w2: "wylęgowe",	pmi: 19.8375
PmiEntry(w1: "jakakolwiek",	w2: "dyskryminacja",	pmi: 19.8375
PmiEntry(w1: "janowice",	w2: "wielkie",	pmi: 19.8375
PmiEntry(w1: "kadmowe",	w2: "wielkogabarytowe",	pmi: 19.8375
```

After removing stopwords:
```
PmiEntry(w1: "aleksandra",	w2: "zelwerowicza",	pmi: 18.4977
PmiEntry(w1: "analogicznych",	w2: "zainteresowaniach",	pmi: 18.4977
PmiEntry(w1: "anonimowych",	w2: "ofiarodawców",	pmi: 18.4977
PmiEntry(w1: "armatura",	w2: "metalowa",	pmi: 18.4977
PmiEntry(w1: "asesorami",	w2: "sądowymi",	pmi: 18.4977
PmiEntry(w1: "auschwitz",	w2: "birkenau",	pmi: 18.4977
PmiEntry(w1: "bagienny",	w2: "wyżynny",	pmi: 18.4977
PmiEntry(w1: "bezzasadnie",	w2: "niedopuszczonemu",	pmi: 18.4977
PmiEntry(w1: "bibułki",	w2: "tissue",	pmi: 18.4977
PmiEntry(w1: "biegłym",	w2: "rewidentom",	pmi: 18.4977
PmiEntry(w1: "bilet",	w2: "pasażerski",	pmi: 18.4977
PmiEntry(w1: "brzeczki",	w2: "miodowej",	pmi: 18.4977
PmiEntry(w1: "budże",	w2: "towym",	pmi: 18.4977
PmiEntry(w1: "bursie",	w2: "hotelu",	pmi: 18.4977
PmiEntry(w1: "butelka",	w2: "miarowa",	pmi: 18.4977
PmiEntry(w1: "butelkach",	w2: "zaopatrzonych",	pmi: 18.4977
PmiEntry(w1: "chemioterapii",	w2: "niestandardowej",	pmi: 18.4977
PmiEntry(w1: "chwytanie",	w2: "odławianie",	pmi: 18.4977
PmiEntry(w1: "ciągniki",	w2: "siodłowe",	pmi: 18.4977
PmiEntry(w1: "ciążą",	w2: "porodem",	pmi: 18.4977
PmiEntry(w1: "cukrem",	w2: "buraczanym",	pmi: 18.4977
PmiEntry(w1: "cyjanoguan",	w2: "cyjanoguanidynę",	pmi: 18.4977
PmiEntry(w1: "damskich",	w2: "męskich",	pmi: 18.4977
PmiEntry(w1: "destylat",	w2: "winny",	pmi: 18.4977
PmiEntry(w1: "destylatu",	w2: "winnego",	pmi: 18.4977
PmiEntry(w1: "dokumentacje",	w2: "hydrologiczne",	pmi: 18.4977
PmiEntry(w1: "dotacją",	w2: "podmiotową",	pmi: 18.4977
PmiEntry(w1: "doświadczalno",	w2: "produkcyjną",	pmi: 18.4977
PmiEntry(w1: "dur",	w2: "brzuszny",	pmi: 18.4977
PmiEntry(w1: "dury",	w2: "rzekome",	pmi: 18.4977
```

Here we have the same PMI for many bigrams becouse those words ocurred only two times in corpus and those two times they were neighbouring each other.
```
PmiEntry(w1: "aleksandra",	w2: "zelwerowicza",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
PmiEntry(w1: "analogicznych",	w2: "zainteresowaniach",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
PmiEntry(w1: "anonimowych",	w2: "ofiarodawców",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
PmiEntry(w1: "armatura",	w2: "metalowa",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
PmiEntry(w1: "asesorami",	w2: "sądowymi",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
PmiEntry(w1: "auschwitz",	w2: "birkenau",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
PmiEntry(w1: "bagienny",	w2: "wyżynny",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
PmiEntry(w1: "bezzasadnie",	w2: "niedopuszczonemu",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
PmiEntry(w1: "bibułki",	w2: "tissue",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
PmiEntry(w1: "biegłym",	w2: "rewidentom",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
PmiEntry(w1: "bilet",	w2: "pasażerski",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
PmiEntry(w1: "brzeczki",	w2: "miodowej",	pmi: 18.4977;   w1: 2, w2: 2, bigram:740296 entry: 2
```

With considered position in bigram:
```
PmiEntry(w1: "adwokatami",	w2: "radcami",	pmi: 18.4977
PmiEntry(w1: "aerotermalną",	w2: "geotermalną",	pmi: 18.4977
PmiEntry(w1: "afo",	w2: "rmu",	pmi: 18.4977
PmiEntry(w1: "akcyjnemu",	w2: "zakładowemu",	pmi: 18.4977
PmiEntry(w1: "aleksandra",	w2: "zelwerowicza",	pmi: 18.4977
PmiEntry(w1: "aleksandrowski",	w2: "brodnicki",	pmi: 18.4977
PmiEntry(w1: "alternatywnie",	w2: "karami",	pmi: 18.4977
PmiEntry(w1: "analogicznych",	w2: "zainteresowaniach",	pmi: 18.4977
PmiEntry(w1: "anestezjologii",	w2: "chirurgii",	pmi: 18.4977
PmiEntry(w1: "anonimowych",	w2: "ofiarodawców",	pmi: 18.4977
PmiEntry(w1: "anteny",	w2: "zbiorczej",	pmi: 18.4977
PmiEntry(w1: "aplikacją",	w2: "patron",	pmi: 18.4977
PmiEntry(w1: "arabskich",	w2: "czystej",	pmi: 18.4977
PmiEntry(w1: "armatura",	w2: "metalowa",	pmi: 18.4977
PmiEntry(w1: "asesorami",	w2: "sądowymi",	pmi: 18.4977
PmiEntry(w1: "auschwitz",	w2: "birkenau",	pmi: 18.4977
PmiEntry(w1: "austrii",	w2: "królestwie",	pmi: 18.4977
PmiEntry(w1: "azotanow",	w2: "dolomit",	pmi: 18.4977
PmiEntry(w1: "bagienny",	w2: "wyżynny",	pmi: 18.4977
PmiEntry(w1: "bakterii",	w2: "drożdży",	pmi: 18.4977
PmiEntry(w1: "bankowości",	w2: "korespondencyjnej",	pmi: 18.4977
PmiEntry(w1: "baraki",	w2: "domki",	pmi: 18.4977
PmiEntry(w1: "bartoszycki",	w2: "braniewski",	pmi: 18.4977
PmiEntry(w1: "barwione",	w2: "powierzchniowo",	pmi: 18.4977
PmiEntry(w1: "bary",	w2: "spowodowanym",	pmi: 18.4977
PmiEntry(w1: "belgii",	w2: "wielkim",	pmi: 18.4977
PmiEntry(w1: "betel",	w2: "szczytno",	pmi: 18.4977
PmiEntry(w1: "bezledach",	w2: "dorohusku",	pmi: 18.4977
PmiEntry(w1: "bezodpływowych",	w2: "porzucone",	pmi: 18.4977
PmiEntry(w1: "bezprawny",	w2: "utrudniał",	pmi: 18.4977
```

Results are not any better than in original case

# 4-5 Log likelihood ratio
Without removing stopwords

```
LlrEntry(w1: "r",	w2: "nr",	llr: 118823.2423
LlrEntry(w1: "w",	w2: "w",	llr: 104986.4111
LlrEntry(w1: "otrzymuje",	w2: "brzmienie",	llr: 99083.8950
LlrEntry(w1: "dz",	w2: "u",	llr: 66106.7732
LlrEntry(w1: "których",	w2: "mowa",	llr: 57828.0304
LlrEntry(w1: "w",	w2: "art",	llr: 52208.2698
LlrEntry(w1: "w",	w2: "ust",	llr: 51447.3140
LlrEntry(w1: "dodaje",	w2: "się",	llr: 43875.7557
LlrEntry(w1: "o",	w2: "których",	llr: 42665.3994
LlrEntry(w1: "mowa",	w2: "w",	llr: 38945.6068
LlrEntry(w1: "minister",	w2: "właściwy",	llr: 37376.0427
LlrEntry(w1: "drodze",	w2: "rozporządzenia",	llr: 37134.6250
LlrEntry(w1: "którym",	w2: "mowa",	llr: 36594.7195
LlrEntry(w1: "stosuje",	w2: "się",	llr: 33329.0475
LlrEntry(w1: "w",	w2: "o",	llr: 33040.6436
LlrEntry(w1: "o",	w2: "w",	llr: 32823.2966
LlrEntry(w1: "rzeczypospolitej",	w2: "polskiej",	llr: 31869.0445
LlrEntry(w1: "z",	w2: "w",	llr: 31331.7492
LlrEntry(w1: "w",	w2: "z",	llr: 31314.6921
LlrEntry(w1: "z",	w2: "dnia",	llr: 30789.5203
LlrEntry(w1: "do",	w2: "spraw",	llr: 30676.4652
LlrEntry(w1: "na",	w2: "podstawie",	llr: 30247.9736
LlrEntry(w1: "w",	w2: "i",	llr: 29116.1735
LlrEntry(w1: "a",	w2: "także",	llr: 28134.3436
LlrEntry(w1: "o",	w2: "którym",	llr: 26993.5163
LlrEntry(w1: "co",	w2: "najmniej",	llr: 25658.6648
LlrEntry(w1: "od",	w2: "dnia",	llr: 23140.3933
LlrEntry(w1: "w",	w2: "na",	llr: 22156.4570
LlrEntry(w1: "zastępuje",	w2: "się",	llr: 21542.6856
LlrEntry(w1: "której",	w2: "mowa",	llr: 21394.6195
```

We can se that stop words and legal language dominates here,

After removing stop words:
```
LlrEntry(w1: "r",	w2: "nr",	llr: 135910.8754
LlrEntry(w1: "otrzymuje",	w2: "brzmienie",	llr: 81857.9947
LlrEntry(w1: "drodze",	w2: "rozporządzenia",	llr: 39435.4322
LlrEntry(w1: "minister",	w2: "właściwy",	llr: 38454.6318
LlrEntry(w1: "rzeczypospolitej",	w2: "polskiej",	llr: 28281.3264
LlrEntry(w1: "samorządu",	w2: "terytorialnego",	llr: 15685.4192
LlrEntry(w1: "następujące",	w2: "zmiany",	llr: 14512.8855
LlrEntry(w1: "późn",	w2: "zm",	llr: 13969.0430
LlrEntry(w1: "ministrem",	w2: "właściwym",	llr: 13266.2416
LlrEntry(w1: "ministra",	w2: "właściwego",	llr: 13199.0599
LlrEntry(w1: "unii",	w2: "europejskiej",	llr: 12501.2961
LlrEntry(w1: "pozbawienia",	w2: "wolności",	llr: 11594.7867
LlrEntry(w1: "spraw",	w2: "wewnętrznych",	llr: 11022.2922
LlrEntry(w1: "zastrzeżeniem",	w2: "ust",	llr: 10952.3337
LlrEntry(w1: "zostały",	w2: "ogłoszone",	llr: 10751.3477
LlrEntry(w1: "opieki",	w2: "zdrowotnej",	llr: 10224.6199
LlrEntry(w1: "działalności",	w2: "gospodarczej",	llr: 9984.0993
LlrEntry(w1: "obrony",	w2: "narodowej",	llr: 9887.6035
LlrEntry(w1: "terytorium",	w2: "rzeczypospolitej",	llr: 9525.4660
LlrEntry(w1: "skarbu",	w2: "państwa",	llr: 9237.6356
LlrEntry(w1: "finansów",	w2: "publicznych",	llr: 9159.3974
LlrEntry(w1: "zasadach",	w2: "określonych",	llr: 8943.6885
LlrEntry(w1: "produktu",	w2: "leczniczego",	llr: 8900.8218
LlrEntry(w1: "straży",	w2: "granicznej",	llr: 8867.0071
LlrEntry(w1: "ubezpieczeń",	w2: "społecznych",	llr: 8837.6244
LlrEntry(w1: "r",	w2: "poz",	llr: 8591.3460
LlrEntry(w1: "niniejszej",	w2: "ustawy",	llr: 8578.8178
LlrEntry(w1: "administracji",	w2: "rządowej",	llr: 8559.4929
LlrEntry(w1: "ochrony",	w2: "środowiska",	llr: 8491.6027
LlrEntry(w1: "papierów",	w2: "wartościowych",	llr: 8444.4881
```

We can see way better results. There ares still some quirks like r /nr  or r/ poz that are leftoverf from showing years.  We don't have rarely occuring things (that occur always together) like names etc.
We could remove them by filtering by word length, but that would possibly discard some valid word colocations.

With considering word word position in bigram:
```
LlrEntry(w1: "r",	w2: "nr",	llr: 138114.3738
LlrEntry(w1: "otrzymuje",	w2: "brzmienie",	llr: 94190.2259
LlrEntry(w1: "drodze",	w2: "rozporządzenia",	llr: 46738.7679
LlrEntry(w1: "minister",	w2: "właściwy",	llr: 42710.2157
LlrEntry(w1: "rzeczypospolitej",	w2: "polskiej",	llr: 38700.6827
LlrEntry(w1: "samorządu",	w2: "terytorialnego",	llr: 19745.8505
LlrEntry(w1: "brzmienie",	w2: "art",	llr: 19286.4583
LlrEntry(w1: "następujące",	w2: "zmiany",	llr: 17592.8594
LlrEntry(w1: "ministra",	w2: "właściwego",	llr: 15928.0789
LlrEntry(w1: "unii",	w2: "europejskiej",	llr: 15739.2066
LlrEntry(w1: "późn",	w2: "zm",	llr: 15124.4550
LlrEntry(w1: "pozbawienia",	w2: "wolności",	llr: 14290.6929
LlrEntry(w1: "ministrem",	w2: "właściwym",	llr: 13902.1048
LlrEntry(w1: "finansów",	w2: "publicznych",	llr: 13148.2616
LlrEntry(w1: "spraw",	w2: "wewnętrznych",	llr: 12961.4770
LlrEntry(w1: "obrony",	w2: "narodowej",	llr: 12866.0413
LlrEntry(w1: "opieki",	w2: "zdrowotnej",	llr: 12597.7115
LlrEntry(w1: "skarbu",	w2: "państwa",	llr: 12576.3057
LlrEntry(w1: "zostały",	w2: "ogłoszone",	llr: 12237.0139
LlrEntry(w1: "działalności",	w2: "gospodarczej",	llr: 12125.9523
LlrEntry(w1: "niniejszej",	w2: "ustawy",	llr: 12124.0952
LlrEntry(w1: "terytorium",	w2: "rzeczypospolitej",	llr: 12113.0204
LlrEntry(w1: "produktu",	w2: "leczniczego",	llr: 11529.6340
LlrEntry(w1: "ubezpieczeń",	w2: "społecznych",	llr: 11489.3977
LlrEntry(w1: "ochrony",	w2: "środowiska",	llr: 11442.5302
LlrEntry(w1: "straży",	w2: "granicznej",	llr: 11040.0537
LlrEntry(w1: "zastrzeżeniem",	w2: "ust",	llr: 10981.7881
LlrEntry(w1: "dz",	w2: "urz",	llr: 10910.8997
LlrEntry(w1: "administracji",	w2: "rządowej",	llr: 10480.1592
LlrEntry(w1: "podlega",	w2: "karze",	llr: 10477.5861
```

We can see little changes bettween trial with ant without considering word order in bigram.

# 6. Answer the following questions:
1. Which measure works better for the problem?
    * In my opinion Log likelihood ratio performed a little better in this task ( when stopwords were removed).
2. What would be needed, besides good measure, to build a dictionary of multiword expressions?
    * big, diverse corpus, and probably lemmatization, because  we have
    ```
    LlrEntry(w1: "znaczącymi",    w2: "inwestorami",    llr: 177.0033
    LlrEntry(w1: "znaczącym",    w2: "inwestorem",    llr: 177.0033
    ```
    which distorts results a little bit

3. Can you identify a certain threshold which clearly divides the good expressions from the bad?
    * I cannot see clear threshold dividing good & bad expressions; bad ones are near the top along with good ones.
    * If i had to choose i would chose llr in vicinity of 100  to be that threshold, because in higher llrs we have sometimes valid multiword expressions like :
    ```
    4400, LlrEntry(w1: "środkami",	w2: "publicznymi",	llr: 212.0803
    4600, LlrEntry(w1: "wyższe",	w2: "studia",	llr: 204.2139
    5200, LlrEntry(w1: "inspektorów",	w2: "sanitarnych",	llr: 183.7519
    5400, LlrEntry(w1: "kwalifikowanym",	w2: "podpisem",	llr: 177.5824
    7200, LlrEntry(w1: "hitlerowskich",	w2: "obozów",	llr: 137.1484
    ```
    * If i had to choose threshold for pmi w would choose pmi = 9 or 10, because on this level we have:
    ```
    21600, PmiEntry(w1: "spółek",	w2: "akcyjnych",	pmi: 10.1287
    22500, PmiEntry(w1: "pobrany",	w2: "podatek",	pmi: 9.9952
    23400, PmiEntry(w1: "ubój",	w2: "sanitarny",	pmi: 9.8666
    26400, PmiEntry(w1: "instytucję",	w2: "finansową",	pmi: 9.4575
    ``` 
    
> Here first column in exceprts is index of bigram in list sored by llr and pmi respectively